In [1]:
import numpy as np
from numpy import linalg

################################################################################
# Function: idempotents                                                        #
#                                                                              #
# Purpose: find principal idempotents of a Hermitian matrix                    #
#                                                                              #
# Arguments:                                                                   #
#   A        Hermitian sage or numpy matrix     adjacency matrix/Hamiltonian   #
#                                                                              #
# Returns: principal idempotents, eigenvalues                                  #
#                                                                              #
################################################################################

def idempotents(A):
    if not matrix(A).is_hermitian():
        print("A is not Hermitian")
        return []
    # Extracts eigenvalues and eigenvectors
    W,V = linalg.eigh(A)
    # Converts eigenvectors into orthonormal basis
    Q, R = linalg.qr(V) 
    Q = matrix(Q)
    # Number of eigenvectors
    A = np.matrix(A)
    n = len(A)
    idems = []
    evals = []
    i = 0
    j = 0
    while i < n:
        # Zero matrix
        E = matrix(n, n, 0)
        # Checks if eigenvectors share an eigenvalue
        while j < n and abs(W[i] - W[j]) < 0.01:
            # Converts eigenvector into column vecror
            v = Q.column(j).column()
            # updates idempotent
            E = E + v*v.conjugate_transpose();
            j = j + 1
        E = E
        idems.append(E)
        evals.append(W[i].round(3))
        i = j
    return idems, evals

In [2]:
################################################################################
# Function: transfer                                                           #
#                                                                              #
# Purpose: return transfer matrix U(t) given adjacency matrix/Hamiltonian      #
#                                                                              #
# Arguments:                                                                   #
#   A        Hermitian sage or numpy matrix     adjacency matrix/Hamiltonian   #
#   t        float                              time                           #
#   k        integer                            rounding accuracy              #
#                                                                              #
# Returns: transfer U(t)                                                       #
#                                                                              #
################################################################################

def transfer(A, t):
    A = matrix(A)
    return matrix(exp(-1*I*t*A))

In [3]:
################################################################################
# Function: transfer_at_time_r                                                 #
#                                                                              #
# Purpose: find real part of transfer matrix at specific time and index        #
#                                                                              #
# Arguments:                                                                   #
#   A        Hermitian sage or numpy matrix     adjacency matrix/Hamiltonian   #
#   i        integer                            row number                     #
#   j        integer                            column number                  #
#                                                                              #
# Returns: real part of U(t) at index [i][j]                                   #
#                                                                              #
################################################################################

def transfer_at_time_r(A,t,i,j):
    U = transfer(A,t)
    if abs(U[i][j].real()) < 0.00001:
        return 0
    return U[i][j].real()

################################################################################
# Function: transfer_at_time_i                                                 #
#                                                                              #
# Purpose: find imaginary part of transfer matrix at specific time and index   #
#                                                                              #
# Arguments:                                                                   #
#   A        Hermitian sage or numpy matrix     adjacency matrix/Hamiltonian   #
#   i        integer                            row number                     #
#   j        integer                            column number                  #
#                                                                              #
# Returns: imaginary part of U(t) at index [i][j]                              #
#                                                                              #
################################################################################

def transfer_at_time_i(A,t,i,j):
    U = transfer(A,t)
    if abs(U[i][j].imag()) < 0.00001:
        return 0
    return U[i][j].imag()

################################################################################
# Function: transfer_at_time_a                                                 #
#                                                                              #
# Purpose: find absolute value of transfer matrix at specific time and index   #
#                                                                              #
# Arguments:                                                                   #
#   A        Hermitian sage or numpy matrix     adjacency matrix/Hamiltonian   #
#   i        integer                            row number                     #
#   j        integer                            column number                  #
#                                                                              #
# Returns: absolute value of U(t) at index [i][j]                              #
#                                                                              #
################################################################################

def transfer_at_time_a(A,t,i,j):
    U = transfer(A,t)
    if U[i][j].abs() < 0.00001:
        return 0
    return U[i][j].abs()

In [4]:
K = np.matrix([[0,1j,-1j,1j,0,0,-1j,0,0],[-1j,0,1j,0,1j,0,0,-1j,0],[1j,-1j,0,0,0,1j,0,0,-1j],\
               [-1j,0,0,0,1j,-1j,1j,0,0],[0,-1j,0,-1j,0,1j,0,1j,0],[0,0,-1j,1j,-1j,0,0,0,1j],\
               [1j,0,0,-1j,0,0,0,1j,-1j],[0,1j,0,0,-1j,0,-1j,0,1j],[0,0,1j,0,0,-1j,1j,-1j,0]]); matrix(K)

[   0.0  1.0*I -1.0*I  1.0*I    0.0    0.0 -1.0*I    0.0    0.0]
[-1.0*I    0.0  1.0*I    0.0  1.0*I    0.0    0.0 -1.0*I    0.0]
[ 1.0*I -1.0*I    0.0    0.0    0.0  1.0*I    0.0    0.0 -1.0*I]
[-1.0*I    0.0    0.0    0.0  1.0*I -1.0*I  1.0*I    0.0    0.0]
[   0.0 -1.0*I    0.0 -1.0*I    0.0  1.0*I    0.0  1.0*I    0.0]
[   0.0    0.0 -1.0*I  1.0*I -1.0*I    0.0    0.0    0.0  1.0*I]
[ 1.0*I    0.0    0.0 -1.0*I    0.0    0.0    0.0  1.0*I -1.0*I]
[   0.0  1.0*I    0.0    0.0 -1.0*I    0.0 -1.0*I    0.0  1.0*I]
[   0.0    0.0  1.0*I    0.0    0.0 -1.0*I  1.0*I -1.0*I    0.0]

In [ ]:
transfer(K,2*np.pi/sqrt(27))[0][4]
# 1 - 1.11022302 × 10-16 i

In [ ]:
EK, lK = idempotents(K)
for i in range(len(lK)):
    print("Idempotent for", lK[i])
    print(EK[i].round(3))

In [ ]:
W,V = linalg.eigh(K); W

In [ ]:
# Plots absolute value of U(t) for matrix K from time 0 to 3pi
PH00 = plot(transfer_at_time_a(K,x,0,0),0,3*np.pi)
PH10 = plot(transfer_at_time_a(K,x,1,0),0,3*np.pi)
PH20 = plot(transfer_at_time_a(K,x,2,0),0,3*np.pi)
PH30 = plot(transfer_at_time_a(K,x,3,0),0,3*np.pi)
PH40 = plot(transfer_at_time_a(K,x,4,0),0,3*np.pi)
PH50 = plot(transfer_at_time_a(K,x,5,0),0,3*np.pi)
PH60 = plot(transfer_at_time_a(K,x,6,0),0,3*np.pi)
PH70 = plot(transfer_at_time_a(K,x,7,0),0,3*np.pi)
PH80 = plot(transfer_at_time_a(K,x,8,0),0,3*np.pi)

PH01 = plot(transfer_at_time_a(K,x,0,1),0,3*np.pi)
PH11 = plot(transfer_at_time_a(K,x,1,1),0,3*np.pi)
PH21 = plot(transfer_at_time_a(K,x,2,1),0,3*np.pi)
PH31 = plot(transfer_at_time_a(K,x,3,1),0,3*np.pi)
PH41 = plot(transfer_at_time_a(K,x,4,1),0,3*np.pi)
PH51 = plot(transfer_at_time_a(K,x,5,1),0,3*np.pi)
PH61 = plot(transfer_at_time_a(K,x,6,1),0,3*np.pi)
PH71 = plot(transfer_at_time_a(K,x,7,1),0,3*np.pi)
PH81 = plot(transfer_at_time_a(K,x,8,1),0,3*np.pi)

PH02 = plot(transfer_at_time_a(K,x,0,2),0,3*np.pi)
PH12 = plot(transfer_at_time_a(K,x,1,2),0,3*np.pi)
PH22 = plot(transfer_at_time_a(K,x,2,2),0,3*np.pi)
PH32 = plot(transfer_at_time_a(K,x,3,2),0,3*np.pi)
PH42 = plot(transfer_at_time_a(K,x,4,2),0,3*np.pi)
PH52 = plot(transfer_at_time_a(K,x,5,2),0,3*np.pi)
PH62 = plot(transfer_at_time_a(K,x,6,2),0,3*np.pi)
PH72 = plot(transfer_at_time_a(K,x,7,2),0,3*np.pi)
PH82 = plot(transfer_at_time_a(K,x,8,2),0,3*np.pi)

PH03 = plot(transfer_at_time_a(K,x,0,3),0,3*np.pi)
PH13 = plot(transfer_at_time_a(K,x,1,3),0,3*np.pi)
PH23 = plot(transfer_at_time_a(K,x,2,3),0,3*np.pi)
PH33 = plot(transfer_at_time_a(K,x,3,3),0,3*np.pi)
PH43 = plot(transfer_at_time_a(K,x,4,3),0,3*np.pi)
PH53 = plot(transfer_at_time_a(K,x,5,3),0,3*np.pi)
PH63 = plot(transfer_at_time_a(K,x,6,3),0,3*np.pi)
PH73 = plot(transfer_at_time_a(K,x,7,3),0,3*np.pi)
PH83 = plot(transfer_at_time_a(K,x,8,3),0,3*np.pi)

PH04 = plot(transfer_at_time_a(K,x,0,4),0,3*np.pi)
PH14 = plot(transfer_at_time_a(K,x,1,4),0,3*np.pi)
PH24 = plot(transfer_at_time_a(K,x,2,4),0,3*np.pi)
PH34 = plot(transfer_at_time_a(K,x,3,4),0,3*np.pi)
PH44 = plot(transfer_at_time_a(K,x,4,4),0,3*np.pi)
PH54 = plot(transfer_at_time_a(K,x,5,4),0,3*np.pi)
PH64 = plot(transfer_at_time_a(K,x,6,4),0,3*np.pi)
PH74 = plot(transfer_at_time_a(K,x,7,4),0,3*np.pi)
PH84 = plot(transfer_at_time_a(K,x,8,4),0,3*np.pi)

PH05 = plot(transfer_at_time_a(K,x,0,5),0,3*np.pi)
PH15 = plot(transfer_at_time_a(K,x,1,5),0,3*np.pi)
PH25 = plot(transfer_at_time_a(K,x,2,5),0,3*np.pi)
PH35 = plot(transfer_at_time_a(K,x,3,5),0,3*np.pi)
PH45 = plot(transfer_at_time_a(K,x,4,5),0,3*np.pi)
PH55 = plot(transfer_at_time_a(K,x,5,5),0,3*np.pi)
PH65 = plot(transfer_at_time_a(K,x,6,5),0,3*np.pi)
PH75 = plot(transfer_at_time_a(K,x,7,5),0,3*np.pi)
PH85 = plot(transfer_at_time_a(K,x,8,5),0,3*np.pi)

PH06 = plot(transfer_at_time_a(K,x,0,6),0,3*np.pi)
PH16 = plot(transfer_at_time_a(K,x,1,6),0,3*np.pi)
PH26 = plot(transfer_at_time_a(K,x,2,6),0,3*np.pi)
PH36 = plot(transfer_at_time_a(K,x,3,6),0,3*np.pi)
PH46 = plot(transfer_at_time_a(K,x,4,6),0,3*np.pi)
PH56 = plot(transfer_at_time_a(K,x,5,6),0,3*np.pi)
PH66 = plot(transfer_at_time_a(K,x,6,6),0,3*np.pi)
PH76 = plot(transfer_at_time_a(K,x,7,6),0,3*np.pi)
PH86 = plot(transfer_at_time_a(K,x,8,6),0,3*np.pi)

PH07 = plot(transfer_at_time_a(K,x,0,7),0,3*np.pi)
PH17 = plot(transfer_at_time_a(K,x,1,7),0,3*np.pi)
PH27 = plot(transfer_at_time_a(K,x,2,7),0,3*np.pi)
PH37 = plot(transfer_at_time_a(K,x,3,7),0,3*np.pi)
PH47 = plot(transfer_at_time_a(K,x,4,7),0,3*np.pi)
PH57 = plot(transfer_at_time_a(K,x,5,7),0,3*np.pi)
PH67 = plot(transfer_at_time_a(K,x,6,7),0,3*np.pi)
PH77 = plot(transfer_at_time_a(K,x,7,7),0,3*np.pi)
PH87 = plot(transfer_at_time_a(K,x,8,7),0,3*np.pi)

PH08 = plot(transfer_at_time_a(K,x,0,8),0,3*np.pi)
PH18 = plot(transfer_at_time_a(K,x,1,8),0,3*np.pi)
PH28 = plot(transfer_at_time_a(K,x,2,8),0,3*np.pi)
PH38 = plot(transfer_at_time_a(K,x,3,8),0,3*np.pi)
PH48 = plot(transfer_at_time_a(K,x,4,8),0,3*np.pi)
PH58 = plot(transfer_at_time_a(K,x,5,8),0,3*np.pi)
PH68 = plot(transfer_at_time_a(K,x,6,8),0,3*np.pi)
PH78 = plot(transfer_at_time_a(K,x,7,8),0,3*np.pi)
PH88 = plot(transfer_at_time_a(K,x,8,8),0,3*np.pi)

In [ ]:
# graphics_array(((PH00,PH01,PH02,PH03,PH04,PH05,PH06,PH07,PH08), \
#                 (PH10,PH11,PH12,PH13,PH14,PH15,PH16,PH17,PH18), \
#                 (PH20,PH21,PH22,PH23,PH24,PH25,PH26,PH27,PH28), \
#                 (PH30,PH31,PH32,PH33,PH34,PH35,PH36,PH37,PH38), \
#                 (PH40,PH41,PH42,PH43,PH44,PH45,PH46,PH47,PH48), \
#                 (PH50,PH51,PH52,PH53,PH54,PH55,PH56,PH57,PH58), \
#                 (PH60,PH61,PH62,PH63,PH64,PH65,PH66,PH67,PH68), \
#                 (PH70,PH71,PH72,PH73,PH74,PH75,PH76,PH77,PH78), \
#                 (PH80,PH81,PH82,PH83,PH84,PH85,PH86,PH87,PH88)))

In [ ]:
graphics_array(((PH00,PH01,PH02),(PH10,PH11,PH12),(PH20,PH21,PH22)))

In [ ]:
PH00

In [ ]:
K3 = np.matrix([[0,1j,-1j],[-1j,0,1j],[1j,-1j,0]]); matrix(K3)

In [ ]:
# Plots absolute value of U(t) for matrix H from time 0 to 3pi
PK300 = plot(transfer_at_time_a(K3,x,0,0),0,3*np.pi)
PK310 = plot(transfer_at_time_a(K3,x,1,0),0,3*np.pi)
PK320 = plot(transfer_at_time_a(K3,x,2,0),0,3*np.pi)
PK301 = plot(transfer_at_time_a(K3,x,0,1),0,3*np.pi)
PK311 = plot(transfer_at_time_a(K3,x,1,1),0,3*np.pi)
PK321 = plot(transfer_at_time_a(K3,x,2,1),0,3*np.pi)
PK302 = plot(transfer_at_time_a(K3,x,0,2),0,3*np.pi)
PK312 = plot(transfer_at_time_a(K3,x,1,2),0,3*np.pi)
PK322 = plot(transfer_at_time_a(K3,x,2,2),0,3*np.pi)

In [ ]:
PK300

In [ ]:
graphics_array(((PK300,PK301,PK302), (PK310,PK311,PK312), (PK320,PK321,PK322)))

In [ ]:
def H_theta(theta):
    return np.matrix([[0,1,1],[1,0,exp(-1j*theta)],[1,exp(1j*theta),0]])

In [ ]:
# Idempotents for H_theta
H = H_theta(np.pi/2)
EH, lH = idempotents(H);
for i in range(len(lH)):
    print("Idempotent for", int(lH[i]))
    print(EH[i].round(3))

In [ ]:
# Plots absolute value of U(t) for matrix H from time 0 to 3pi
PaH00 = plot(transfer_at_time_a(H,x,0,0),0,3*np.pi)
PaH10 = plot(transfer_at_time_a(H,x,1,0),0,3*np.pi)
PaH20 = plot(transfer_at_time_a(H,x,2,0),0,3*np.pi)
PaH01 = plot(transfer_at_time_a(H,x,0,1),0,3*np.pi)
PaH11 = plot(transfer_at_time_a(H,x,1,1),0,3*np.pi)
PaH21 = plot(transfer_at_time_a(H,x,2,1),0,3*np.pi)
PaH02 = plot(transfer_at_time_a(H,x,0,2),0,3*np.pi)
PaH12 = plot(transfer_at_time_a(H,x,1,2),0,3*np.pi)
PaH22 = plot(transfer_at_time_a(H,x,2,2),0,3*np.pi)

In [ ]:
graphics_array(((PaH00,PaH01,PaH02), (PaH10,PaH11,PaH12), (PaH20,PaH21,PaH22)))

In [ ]:
# Plots real part of U(t) for matrix H from time 0 to 3pi
PrH00 = plot(transfer_at_time_r(H,x,0,0),0,3*np.pi)
PrH10 = plot(transfer_at_time_r(H,x,1,0),0,3*np.pi)
PrH20 = plot(transfer_at_time_r(H,x,2,0),0,3*np.pi)
PrH01 = plot(transfer_at_time_r(H,x,0,1),0,3*np.pi)
PrH11 = plot(transfer_at_time_r(H,x,1,1),0,3*np.pi)
PrH21 = plot(transfer_at_time_r(H,x,2,1),0,3*np.pi)
PrH02 = plot(transfer_at_time_r(H,x,0,2),0,3*np.pi)
PrH12 = plot(transfer_at_time_r(H,x,1,2),0,3*np.pi)
PrH22 = plot(transfer_at_time_r(H,x,2,2),0,3*np.pi)

In [ ]:
graphics_array(((PrH00,PrH01,PrH02), (PrH10,PrH11,PrH12), (PrH20,PrH21,PrH22)))

In [ ]:
# Plots imaginary part of U(t) for matrix H from time 0 to 3pi
PiH00 = plot(transfer_at_time_i(H,x,0,0),0,3*np.pi)
PiH10 = plot(transfer_at_time_i(H,x,1,0),0,3*np.pi)
PiH20 = plot(transfer_at_time_i(H,x,2,0),0,3*np.pi)
PiH01 = plot(transfer_at_time_i(H,x,0,1),0,3*np.pi)
PiH11 = plot(transfer_at_time_i(H,x,1,1),0,3*np.pi)
PiH21 = plot(transfer_at_time_i(H,x,2,1),0,3*np.pi)
PiH02 = plot(transfer_at_time_i(H,x,0,2),0,3*np.pi)
PiH12 = plot(transfer_at_time_i(H,x,1,2),0,3*np.pi)
PiH22 = plot(transfer_at_time_i(H,x,2,2),0,3*np.pi)

In [ ]:
graphics_array(((PiH00,PiH01,PiH02), (PiH10,PiH11,PiH12), (PiH20,PiH21,PiH22)))

In [ ]:
#comment here#